In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, ReLU, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical


In [41]:
import cv2
import numpy as np

In [42]:
from tensorflow.keras import backend as K


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [43]:
SIZE = 256
N = 100

In [44]:
# X = np.zeros((N, SIZE, SIZE))
# A = np.zeros((N, SIZE, SIZE))
# B = np.zeros((N, SIZE, SIZE))
# for i in range(1, N + 1):
#     X[i - 1] = cv2.imread('data/bnw/' + str(i + 7) + '.jpeg', 0)
#     A[i - 1] = cv2.imread('data/a/' + str(i + 7) + '.jpeg', 0)
#     B[i - 1] = cv2.imread('data/b/' + str(i + 7) + '.jpeg', 0)



In [45]:
# Data loader
class LoadData(Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, img_paths, a_paths, bins):
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_paths = img_paths
        self.a_paths = a_paths
        self.bins = bins
        self.classes = self.bins.shape[0] - 1

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.img_paths[i: i + self.batch_size]
        batch_a_paths = self.a_paths[i: i + self.batch_size]
        x = np.zeros((self.batch_size, self.img_size, self.img_size), dtype="float32")

        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, color_mode='grayscale')
            x[j] = img
        a = np.zeros((self.batch_size, self.img_size, self.img_size), dtype="uint8")
        for j, path in enumerate(batch_a_paths):
            img = load_img(path, color_mode="grayscale")
            a[j] = img

        x = x / 255
        a_binned = np.digitize(a, self.bins) - 1
        a_hot = to_categorical(a_binned, num_classes=self.classes)
        output_shape = (self.batch_size, self.img_size, self.img_size, 1)

        return x.reshape(output_shape), a_hot

In [46]:
bins_list = [0, 64, 96, 112, 120, 128, 136, 144, 160, 192, 256]
num_bins = len(bins_list) - 1
bins = np.array(bins_list)
# a_binned = np.digitize(A, bins) - 1
# b_binned = np.digitize(B, bins) - 1

In [47]:
# One hot encode A and B

# a_hot = to_categorical(a_binned)
# b_hot = to_categorical(b_binned)

In [48]:
# X = X.reshape(N, SIZE, SIZE, 1)
# y = np.stack((A, B)).transpose((1, 2, 3, 0))

In [49]:
# Normalize the data
# X = X / 255
# y = y / 255

Following are utility functions for creating a U-net Model

In [50]:
def conv_stack(input_layer, filters):
    conv1 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(input_layer)
    batch_norm1 = BatchNormalization()(conv1)
    relu1 = ReLU()(batch_norm1)

    conv2 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(relu1)
    batch_norm2 = BatchNormalization()(conv2)
    relu2 = ReLU()(batch_norm2)

    return relu2

In [51]:
def encoder_block(input_layer, filters):
    conv = conv_stack(input_layer, filters)
    max_pool = MaxPooling2D(pool_size=(2, 2))(conv)

    return conv, max_pool

In [52]:
def decoder_block(input_layer, skip_layer, filters):
    up = Conv2DTranspose(filters, 2, strides=2, padding='same')(input_layer)
    conc = Concatenate()([up, skip_layer])
    dec = conv_stack(conc, filters)

    return dec

In [53]:
def get_model(size, init_filters):
    inputs = Input((size, size, 1))

    conv1, max_pool1 = encoder_block(inputs, init_filters)
    conv2, max_pool2 = encoder_block(max_pool1, init_filters * 2)
    conv3, max_pool3 = encoder_block(max_pool2, init_filters * 4)

    middle_block = conv_stack(max_pool3, init_filters * 8)

    decoder1 = decoder_block(middle_block, conv3, init_filters * 4)
    decoder2 = decoder_block(decoder1, conv2, init_filters * 2)
    decoder3 = decoder_block(decoder2, conv1, init_filters)

    # soft = Softmax(axis=1)(decoder3)
    output_a = Conv2D(num_bins, 1, padding='same', activation='softmax')(decoder3)
    # output_b = Conv2D(32, 1, padding='same', activation='softmax')(decoder3)

    model = Model(inputs, output_a)
    return model

In [54]:
model_a = get_model(SIZE, 64)
# model_b = get_model(SIZE, 64)
# model.summary()

In [55]:
# Load Model
# model_a.load_weights("aero-color_a_v2.h5")
# model_b.load_weights("aero-color_b_v2.h5")

In [56]:
# weight_a = [276.4182377999916, 10.096705660138996, 2.160336840606263, 1.8126515533081804, 0.3854523724941714,
#             0.22361807604925382, 1.040882132359878, 1.6375415188587383, 4.57322774863995, 37.81061231083443]

weight_a = [50.4182377999916, 5.096705660138996, 1.160336840606263, 1.1126515533081804, 0.3854523724941714,
            0.22361807604925382, 1.040882132359878, 1.1375415188587383, 2.57322774863995, 10.81061231083443]

# weight_b = [263.46130653, 6.29587218, 2.67742605, 2.33413422, 0.63087839,
#             0.31916905, 0.66331561, 0.56931914, 1.04141953, 10.7601525]
weight_b = [23.46130653, 2.29587218, 1.67742605, 1.13413422, 0.63087839,
            0.31916905, 0.66331561, 0.76931914, 0.84141953, 2.7601525]
model_a.compile(optimizer=Adam(0.01), loss=weighted_categorical_crossentropy(weights=weight_a), metrics='accuracy')

In [57]:
import os

# Make data generator
IMAGE_DIR = 'data/bnw/'
A_DIR = 'data/a/'
B_DIR = 'data/b/'

img_paths = sorted(
    [
        os.path.join(IMAGE_DIR, fname)
        for fname in os.listdir(IMAGE_DIR)
        if fname.endswith(".jpeg")
    ]
)
a_paths = sorted(
    [
        os.path.join(A_DIR, fname)
        for fname in os.listdir(A_DIR)
        if fname.endswith(".jpeg") and not fname.startswith(".")
    ]
)

b_paths = sorted(
    [
        os.path.join(B_DIR, fname)
        for fname in os.listdir(B_DIR)
        if fname.endswith(".jpeg") and not fname.startswith(".")
    ]
)


In [58]:
import random

total_samples = N

random.Random(42).shuffle(img_paths)
random.Random(42).shuffle(a_paths)
random.Random(42).shuffle(b_paths)

img_paths = img_paths[:total_samples]
a_paths = a_paths[:total_samples]
# b_paths = b_paths[:total_samples]

batch_size = 4

load_data = LoadData(
    batch_size, SIZE, img_paths, a_paths, bins
)

10


In [59]:
# Train Model
callbacks = [
    ModelCheckpoint("aero-color_a_test.h5", save_best_only=True, save_weights_only=True, monitor='loss')
]

model_a.fit(load_data, epochs=5, verbose=1, batch_size=4, callbacks=callbacks)

(4, 256, 256, 10)
Epoch 1/5
(4, 256, 256, 10)
(4, 256, 256, 10)
 3/25 [==>...........................] - ETA: 1:59 - loss: 1.0187 - accuracy: 0.3782(4, 256, 256, 10)


KeyboardInterrupt: 

In [ ]:
X = cv2.imread('data/bnw/' + str(7) + '.jpeg', 0)
a = cv2.imread('data/a/' + str(7) + '.jpeg', 0)
b = cv2.imread('data/b/' + str(7) + '.jpeg', 0)

In [ ]:
INDEX = 50

In [ ]:
a_hats = model_a.predict(X.reshape(1, SIZE, SIZE, 1))
# b_hats = model_b.predict(X[INDEX].reshape(1, SIZE, SIZE, 1))

In [ ]:
# print(np.sum(a_hats[0, :, 0, 0]))
# print(np.sum(a_hats[0, 0, :, 0]))
# print(np.sum(a_hats[0, 0, 0, :]))

In [ ]:
a_hats = np.argmax(a_hats, axis=3)
# b_hats = np.argmax(b_hats, axis=3)
# b_hats = np.argmax(b_hats, axis=3)

In [ ]:
# Real Image
# L = X[INDEX].reshape((SIZE, SIZE)) * 255
# a = y[INDEX, :, :, 0] * 255
# b = y[INDEX, :, :, 1] * 255
# lab = np.array([L, a, b]).transpose((1, 2, 0)).astype('uint8')
# img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
# cv2.imwrite('001.jpeg', img)
#

In [ ]:
# # Real Image Binned
# L = X[INDEX].reshape((SIZE, SIZE)) * 255
# a2 = a_binned[INDEX, :, :]
# b2 = b_binned[INDEX, :, :]
# a2 = bins[a2] + 0.5 * (bins[a2 + 1] - bins[a2])
# b2 = bins[b2] + 0.5 * (bins[b2 + 1] - bins[b2])
# lab = np.array([L, a2, b2]).transpose((1, 2, 0)).astype('uint8')
# img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
# cv2.imwrite('002.jpeg', img)

In [ ]:
L = X[INDEX].reshape((SIZE, SIZE)) * 255
a_hat = a_hats[0, :, :]
# b_hat = b[0, :, :]
# a_hat = bins[a_hat] + 0.5 * (bins[a_hat + 1] - bins[a_hat])
# b_hat = bins[b_hat] + 0.5 * (bins[b_hat + 1] - bins[b_hat])
lab = np.array([L, a_hat, b]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('003.jpeg', img)

In [ ]:
# unique, counts = np.unique(a_binned, return_counts=True)
# counts = counts / counts.sum()
# # weights_a = counts.sum() / counts / num_bins
# print(dict(zip(unique, counts)))

In [ ]:
cv2.imwrite('a_our.jpeg', a_hat)
cv2.imwrite('a.jpeg', a)
# cv2.imwrite('b_our.jpeg', b_hat)
# cv2.imwrite('b.jpeg', b2)